In [104]:
import pandas as pd
import numpy as np
import nltk
from collections import defaultdict
from collections import Counter
from src.classes.database.offer import MongoOffer
from tqdm import tqdm
from enum import Enum
import re
import json

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [266]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()]
tagged = nltk.pos_tag(token_list[0])

direction = Enum('direction', ['FORWARD', 'BACKWARD', 'SELF'])

In [267]:
def get_all_words(token_list: list[list]) -> list:
    words = []
    for sentence in token_list:
        for word in sentence:
            words.append(word)
    return sorted(set(words))

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def p_next_word(given: str, looking: str, look_ahead: int, tokens: list[list]) -> float:
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
    return nwo[looking] / sum(nwo.values()) if nwo[looking] != 0 else 0

# Old method, slow and not used anymore
def calculate_all_probabilities(tokens: list[list], look_ahead) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = defaultdict()
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in tqdm(p_word):
        p_word[word] = [defaultdict(float) for x in range(0,look_ahead)]
        for x in range(0,look_ahead):
            for sub_word in all_words:
                p = p_next_word(word, sub_word, x+1, tokens)
                if p > 0:
                    p_word[word][x][sub_word] = p_next_word(word, sub_word, x+1, tokens)
                #print(f"{word} | {str(x)} | {sub_word}: {p_word[word][x][sub_word]}")
    return p_word
#data = calculate_all_probabilities(token_list, 1)

## HMaxtrix
Used for storing all word occurrences in a 3 dimensional matrix

### Data we can get from matrix
 1. p(start, find) = probability that 'find' is a n-order forward word for 'start' | single cell value / sum of 'start' row
 2. p(start, find) = probability that 'find' is a n-order backward word for 'start' | single cell value / sum of 'start' column
 3. p(row_word) = probability that 'row word' will be a n-order forward word | sum of row / entire table sum
 4. p(col_word) = probability that 'col word' will be a n-order backward word | sum of column / entire table sum
 5. p(word, order) = probability that 'word' will be n-order word compared to the other orders| single cell value at order / all order values of that word tallied (through the table)
 6.  p(word-row, order) = probability that 'word' will be n-order forward word | all of word-row order sum / all order values of that word-row
 7. p(col-row, order) = probability that 'word' will be n-order backward word | all of word-col order sum / all order values of that word-col

In [339]:
class HMatrix:
    def __init__(self) -> None:
        self.labels = None
        self.reverse_labels = None
        self.order = 0
        self.matrix = None

    def p_row_word(self, order: int, row_word: str, word: str) -> float:
        if row_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        row_label = self.reverse_labels[row_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, row_label, :][word_label]
        divisor = sum(self.matrix[order, row_label, :])

        return dividend/divisor if divisor > 0 else 0

    def p_col_word(self, order: int, col_word: str, word: str) -> float:
        if col_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        col_label = self.reverse_labels[col_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, :, col_label][word_label]
        divisor = sum(self.matrix[order, :, col_label])

        return dividend/divisor if divisor > 0 else 0


    def create_matrix(self, tokens: list[list], order: int):
        #Setup
        self.labels = get_all_words(token_list)
        self.reverse_labels = {self.labels[x]: x for x in range(0, len(self.labels))}
        self.order = order
        self.matrix = np.zeros((order, len(self.labels), len(self.labels)))

        #Iteration
        for sentence in tqdm(tokens):
            for x in range(0, len(sentence)):
                word = sentence[x]
                print(f"Word: {word} ({str(self.reverse_labels[word])})")
                for y in range(0, self.order):
                    if x+y+1 < len(sentence):
                        print(f"Lookahead {str(y+1)}: {sentence[x+y+1]} ({str(self.reverse_labels[sentence[x+y+1]])})")
                        self.matrix[y][self.reverse_labels[word]][self.reverse_labels[sentence[x+y+1]]] += 1

In [340]:
matrix = HMatrix()
matrix.create_matrix(token_list, 5)

100%|██████████| 2532/2532 [00:00<00:00, 17728.86it/s]

Word: logitech (1792)
Lookahead 1: g (1381)
Lookahead 2: pro (2212)
Lookahead 3: x (2956)
Lookahead 4: superlight (2633)
Word: g (1381)
Lookahead 1: pro (2212)
Lookahead 2: x (2956)
Lookahead 3: superlight (2633)
Word: pro (2212)
Lookahead 1: x (2956)
Lookahead 2: superlight (2633)
Word: x (2956)
Lookahead 1: superlight (2633)
Word: superlight (2633)
Word: nintendo (2000)
Lookahead 1: switch (2650)
Lookahead 2: lite (1781)
Lookahead 3: gray (1488)
Lookahead 4: with (2941)
Lookahead 5: original (2073)
Word: switch (2650)
Lookahead 1: lite (1781)
Lookahead 2: gray (1488)
Lookahead 3: with (2941)
Lookahead 4: original (2073)
Lookahead 5: box (939)
Word: lite (1781)
Lookahead 1: gray (1488)
Lookahead 2: with (2941)
Lookahead 3: original (2073)
Lookahead 4: box (939)
Lookahead 5: extras (1294)
Word: gray (1488)
Lookahead 1: with (2941)
Lookahead 2: original (2073)
Lookahead 3: box (939)
Lookahead 4: extras (1294)
Lookahead 5: anbernic (786)
Word: with (2941)
Lookahead 1: original (2073)
Loo

## Sentence Class

Input a sentence, it will then use the HMatrix to see which path results the most likely outcome for products

In [341]:
class Sentence:
    def __init__(self, matrix: HMatrix, tokenizer, raw_sentence: str):
        self.raw_sentence = tokenizer.tokenize(raw_sentence.lower())
        self.sentence = []

        # Creating all the blank word classes
        for x, word in enumerate(self.raw_sentence):
            self.sentence.append(Word(word, x))

        # Creating all the forward connections
        for x, word in enumerate(self.raw_sentence):
            print(f"--- {word} ---")
            for neighbor in self.sentence:
                mapped_pos = self.map_word_pos_to_order(neighbor.position-x)

                if mapped_pos[1] is not direction.SELF and mapped_pos[0] < matrix.order: #Looking at itself and order is within scope
                    if mapped_pos[1] is direction.FORWARD:
                        print(f"{'  '*mapped_pos[0]}"
                              f"{word} "
                              f"-{str(mapped_pos[0])}-> "
                              f"{neighbor.key} "
                              f"({str(matrix.p_row_word(mapped_pos[0], word, neighbor.key))})")
                    else:
                        print(f"{'  '*mapped_pos[0]}"
                              f"{word} "
                              f"<-{str(mapped_pos[0])}- "
                              f"{neighbor.key} "
                              f"({str(matrix.p_col_word(mapped_pos[0], word, neighbor.key))})")

                # if word == neighbor.key and x == neighbor.position:
                #     word.forward_neighbors.append({'word': neighbor.key, 'p_forward': -1})
                # else:
                #     word.forward_neighbors.append({'word': neighbor.key, 'p_forward': matrix.p_row_word(neighbor.pos, word, neighbor.key)})
                #
    def map_word_pos_to_order(self, position) -> tuple:

        #Forward word positions
        if position > 0:
            return position-1, direction.FORWARD

        # Backwards word positions
        if position < 0:
            return abs(position)-1, direction.BACKWARD

        #Pos looking at self
        if position == 0:
            return -1, direction.SELF



class Word:
    def __init__(self, key: str, position: int):
        self.key = key
        self.position = position
        self.neighbors = []

    def __str__(self):
        return f"Word: {self.key} | Position: {str(self.position)}"


In [345]:
test = Sentence(matrix, tokenizer, "logitech g pro x superlight intel i7 8700k donny1 donny2 donny3")

--- logitech ---
logitech -0-> g (0.2564102564102564)
  logitech -1-> pro (0.2631578947368421)
    logitech -2-> x (0.16216216216216217)
      logitech -3-> superlight (0.12)
        logitech -4-> intel (0.0)
--- g ---
g <-0- logitech (0.15151515151515152)
g -0-> pro (0.1282051282051282)
  g -1-> x (0.0967741935483871)
    g -2-> superlight (0.05263157894736842)
      g -3-> intel (0.0)
        g -4-> i7 (0.0)
--- pro ---
  pro <-1- logitech (0.05)
pro <-0- g (0.0398406374501992)
pro -0-> x (0.029166666666666667)
  pro -1-> superlight (0.013824884792626729)
    pro -2-> intel (0.0)
      pro -3-> i7 (0.0)
        pro -4-> 8700k (0.0)
--- x ---
    x <-2- logitech (0.09523809523809523)
  x <-1- g (0.08450704225352113)
x <-0- pro (0.0875)
x -0-> superlight (0.041666666666666664)
  x -1-> intel (0.0)
    x -2-> i7 (0.0)
      x -3-> 8700k (0.0)
        x -4-> donny1 (0.0)
--- superlight ---
      superlight <-3- logitech (0.6)
    superlight <-2- g (0.5)
  superlight <-1- pro (0.5)
superl

In [21]:
test.matrix[0, :, 423][1994]

1.0